# Update Interface Example from CSV 


### Model-Specific Code (Do Not Modify)

This section contains code that is specific to the system model. It is updated only when the model is changed and should not require user modifications under normal circumstances.

If a new model is introduced, ensure this section is reviewed and updated as needed.


In [1]:
#!pip install --upgrade git+https://github.com/tkSDISW/Capella_Tools 
import capellambse.decl

from capella_tools import capellambse_helper


from IPython import display as diag_display
resources = {
    "Interface_Control_Example": "Interface_Control_Example/Interface_Control_Example/Interface_Control_Example",
}
path_to_model = "../Interface_Control_Example.aird"
model = capellambse.MelodyModel(path_to_model, resources=resources)


Find interface

In [2]:
for obj in model.la.component_exchanges:  
#    print(obj)
    if obj.name == "hss_hsss_to_agss" :
        interface = obj
        print("Found:" , obj.name, "uuid:",obj.uuid)


Found: hss_hsss_to_agss uuid: 304dad2a-1d26-46f3-a8d4-b3ae841c571b


Locate the the .csv

In [3]:
from __future__ import annotations
import os
import sys
import logging
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, Optional, Dict, List, Tuple

import pandas as pd


# -------- Logging -------------------------------------------------------------

def setup_interface_logger(level: int = logging.INFO) -> logging.Logger:
    """
    Configure and return a logger for the interface loader.
    Call once per notebook (you can re-call to change level).
    """
    logger = logging.getLogger("interfaces")
    logger.setLevel(level)
    # Avoid duplicate handlers when re-running cells
    if not any(isinstance(h, logging.StreamHandler) for h in logger.handlers):
        h = logging.StreamHandler(stream=sys.stdout)
        fmt = logging.Formatter("[%(levelname)s] %(message)s")
        h.setFormatter(fmt)
        logger.addHandler(h)
    # Also quiet down pandas a bit when in DEBUG
    logging.getLogger("pandas").setLevel(logging.WARNING)
    return logger


log = setup_interface_logger(logging.INFO)  # default; set to DEBUG during troubleshooting


# -------- Directory search ----------------------------------------------------

def find_first_matching_dir(
    root: str | Path,
    name_parts: Iterable[str],
    *,
    recurse: bool = False,
    case_insensitive: bool = True,
    logger: logging.Logger = log,
) -> Path:
    """
    Return the first directory under `root` whose name contains any of the given
    `name_parts` substrings (checked in the order provided). If `recurse` is
    False, only scans immediate children; otherwise uses os.walk.
    Raises FileNotFoundError if nothing matches.
    """
    root = Path(root).expanduser().resolve()
    logger.info(f"Searching under root: {root}")
    if not root.is_dir():
        raise FileNotFoundError(f"Root path not found or not a directory: {root}")

    parts = list(name_parts)
    if not parts:
        raise ValueError("Provide at least one name substring in `name_parts`.")

    def norm(s: str) -> str:
        return s.lower() if case_insensitive else s

    if not recurse:
        subdirs = [p for p in root.iterdir() if p.is_dir()]
    else:
        subdirs = []
        for dirpath, dirnames, _ in os.walk(root):
            for d in dirnames:
                subdirs.append(Path(dirpath) / d)

    logger.debug(f"Found {len(subdirs)} subdirectories to scan (recurse={recurse}).")

    # Respect caller's order of name_parts: first part with any match wins
    for part in parts:
        npart = norm(part)
        logger.info(f"Trying name part: '{part}'")
        matches = [d for d in subdirs if npart in norm(d.name)]
        logger.debug(f"Matches for '{part}': {[m.name for m in matches]}")
        if matches:
            chosen = matches[0]
            logger.info(f"Selected directory: {chosen}")
            return chosen

    tried = ", ".join(parts)
    raise FileNotFoundError(
        f"No directory under {root} matched any of: {tried}\n"
        f"Scanned {'recursively' if recurse else 'top-level'} {len(subdirs)} dirs."
    )


# -------- CSV loader objects --------------------------------------------------

@dataclass
class InterfaceCSV:
    """Typed wrapper over a CSV with columns:
       source_index, title, section, property_name, property_value
    """
    path: Path
    df: pd.DataFrame
    logger: logging.Logger = log

    @classmethod
    def load(cls, path: str | Path, *, logger: logging.Logger = log) -> "InterfaceCSV":
        p = Path(path)
        logger.info(f"Loading CSV: {p}")
        if not p.is_file():
            raise FileNotFoundError(f"CSV not found: {p}")

        try:
            df = pd.read_csv(p)
        except Exception as e:
            logger.error(f"Failed to read CSV '{p}': {e}")
            raise

        logger.info(f"Loaded {len(df)} rows, columns: {list(df.columns)}")

        # Normalize column names we rely on
        expected = ("title", "section", "property_name", "property_value")
        for col in expected:
            if col not in df.columns:
                logger.warning(f"Column '{col}' missing in {p.name}; creating with None")
                df[col] = None

        # Required columns check
        required = {"property_name", "property_value"}
        missing = required - set(map(str, df.columns))
        if missing:
            msg = f"{p.name} missing required columns: {sorted(missing)}"
            logger.error(msg)
            raise ValueError(msg)

        # Basic cleanups & dtype expectations
        df["property_name"] = df["property_name"].astype(str)
        # Keep property_value as object; caller may cast later as needed

        ic = cls(path=p, df=df, logger=logger)
        ic._log_quick_peek()
        problems = ic.validate(raise_on_error=False)
        if problems:
            logger.warning("Validation warnings:\n  - " + "\n  - ".join(problems))
        return ic

    # ---- Diagnostics

    def _log_quick_peek(self, n: int = 5) -> None:
        try:
            self.logger.debug("Quick peek (head):")
            self.logger.debug("\n" + str(self.df.head(n)))
            self.logger.debug("Unique sections: " + str(self.df["section"].dropna().unique().tolist()))
            self.logger.debug("Unique titles (first 3): " + str(self.df["title"].dropna().unique().tolist()[:3]))
        except Exception as e:
            self.logger.debug(f"Quick peek skipped: {e}")

    def debug_summary(self, n_titles: int = 5) -> Dict[str, object]:
        """Return a summary dict you can print or inspect in a cell."""
        return {
            "path": str(self.path),
            "rows": len(self.df),
            "columns": list(self.df.columns),
            "null_counts": self.df.isna().sum().to_dict(),
            "section_counts": self.df["section"].value_counts(dropna=False).to_dict(),
            "sample_titles": self.df["title"].dropna().unique().tolist()[:n_titles],
            "sample_props": self.df["property_name"].dropna().unique().tolist()[:10],
        }

    def validate(self, *, raise_on_error: bool = True) -> List[str]:
        """Light validation; returns list of problems and optionally raises."""
        problems: List[str] = []
        # Empty?
        if self.df.empty:
            problems.append("CSV has zero rows.")

        # property_name must exist and be non-empty strings
        if self.df["property_name"].isna().any():
            problems.append("Some rows have NaN in 'property_name'.")

        # Duplicates per (title, property_name, section) can be suspicious
        if all(c in self.df.columns for c in ("title", "section", "property_name")):
            dup_mask = self.df.duplicated(subset=["title", "section", "property_name"], keep=False)
            dup_count = int(dup_mask.sum())
            if dup_count:
                problems.append(f"Found {dup_count} duplicate rows by (title, section, property_name).")

        # Warn if many missing values
        if self.df["property_value"].isna().mean() > 0.25:
            ratio = round(float(self.df["property_value"].isna().mean()), 3)
            problems.append(f"High NaN ratio in 'property_value': {ratio}")

        if raise_on_error and problems:
            raise ValueError("Validation errors:\n  - " + "\n  - ".join(problems))
        return problems

    # ---- Convenience queries

    def property_names(self, *, section: Optional[str] = None, title: Optional[str] = None) -> List[str]:
        q = self.df
        if section is not None:
            q = q[q["section"] == section]
        if title is not None:
            q = q[q["title"] == title]
        names = sorted(q["property_name"].dropna().unique().tolist())
        self.logger.debug(f"property_names(section={section}, title={title}) -> {names}")
        return names

    def properties_for_title(
        self,
        title: str,
        *,
        section: Optional[str] = None,
        drop_na: bool = True,
    ) -> Dict[str, object]:
        q = self.df[self.df["title"] == title]
        if section is not None:
            q = q[q["section"] == section]
        if drop_na:
            q = q[~q["property_value"].isna()]
        result = dict(zip(q["property_name"], q["property_value"]))
        self.logger.debug(f"properties_for_title(title={title!r}, section={section}) -> {len(result)} items")
        return result

    from typing import List, Tuple, Optional
    import pandas as pd
    
    def values_by_property(
        self,
        prop_name: str,
        *,
        section: Optional[str] = None,
        null_str: str = "",
    ) -> List[Tuple[str, str]]:
        q = self.df.loc[self.df["property_name"] == prop_name].copy()
        if section is not None:
            q = q.loc[q["section"] == section]
    
        def to_str_or_null(x):
            # Treat real nulls and common "nullish" text as null_str
            if pd.isna(x):
                return null_str
            if isinstance(x, str) and x.strip().lower() in {"", "nan", "<na>", "none", "null"}:
                return null_str
            return str(x)
    
        titles = q["title"].map(to_str_or_null)
        values = q["property_value"].map(to_str_or_null)
    
        pairs = list(zip(titles.tolist(), values.tolist()))
        self.logger.debug(
            f"values_by_property({prop_name!r}, section={section}) -> {len(pairs)} rows"
        )
        return pairs


# -------- Orchestrator --------------------------------------------------------

@dataclass
class InterfacesBundle:
    base_dir: Path
    process: InterfaceCSV
    structural: InterfaceCSV


def load_interfaces_from_first_match(
    root: str | Path,
    dir_name_parts: Iterable[str],
    *,
    recurse: bool = False,
    case_insensitive: bool = True,
    process_filenames: Iterable[str] = ("process_interface_items.csv", "process_interface.csv"),
    structural_filenames: Iterable[str] = ("structural_interface_items.csv", "structural_interface.csv"),
    dry_run: bool = False,
    logger: logging.Logger = log,
) -> InterfacesBundle:
    """
    1) Find first directory under `root` whose name contains any `dir_name_parts`.
    2) Load the first existing CSV for each category (process/structural) using
       the provided filename candidates (in order).
    3) Return typed wrappers with convenience query methods.
    """
    base = find_first_matching_dir(
        root, dir_name_parts, recurse=recurse, case_insensitive=case_insensitive, logger=logger
    )

    def resolve_one(kind: str, candidates: Iterable[str]) -> Path:
        logger.info(f"Looking for {kind} CSV among candidates: {list(candidates)}")
        for name in candidates:
            p = base / name
            logger.debug(f"Check: {p}")
            if p.is_file():
                logger.info(f"{kind} CSV selected: {p}")
                return p
        raise FileNotFoundError(
            f"{kind}: none of the candidate files found in {base}:\n  - " + "\n  - ".join(map(str, candidates))
        )

    process_path = resolve_one("process", process_filenames)
    structural_path = resolve_one("structural", structural_filenames)

    if dry_run:
        logger.info(f"[dry_run=True] Would load:\n  process:    {process_path}\n  structural: {structural_path}")
        # Load anyway to allow validation previews, but caller knows it's a dry run
    process = InterfaceCSV.load(process_path, logger=logger)
    structural = InterfaceCSV.load(structural_path, logger=logger)

    logger.info("Bundle loaded successfully.")
    return InterfacesBundle(base_dir=base, process=process, structural=structural)

# Turn on detailed logging while you debug:
setup_interface_logger(logging.DEBUG)

bundle = load_interfaces_from_first_match(
    root="per_source_interface_items",
    dir_name_parts=[interface.name],  # tried in order
    recurse=True,
    dry_run=False,  # set True to preview discovery without relying on results downstream
)

# Quick summaries (print or just evaluate in a notebook cell)
bundle.process.debug_summary()
bundle.structural.debug_summary()

# Force validation errors to raise (useful in tests)
bundle.structural.validate(raise_on_error=True)



[INFO] Searching under root: /home/simcenter/studio/Interface_Control_Example/Interface_Control_Example/Notebooks/per_source_interface_items
[DEBUG] Found 6 subdirectories to scan (recurse=True).
[INFO] Trying name part: 'hss_hsss_to_agss'
[DEBUG] Matches for 'hss_hsss_to_agss': ['0000_hss_hsss_to_agss_high_pressure_helium_makeup_for_agss_charge_discharge_buffers']
[INFO] Selected directory: /home/simcenter/studio/Interface_Control_Example/Interface_Control_Example/Notebooks/per_source_interface_items/0000_hss_hsss_to_agss_high_pressure_helium_makeup_for_agss_charge_discharge_buffers
[INFO] Looking for process CSV among candidates: ['process_interface_items.csv', 'process_interface.csv']
[DEBUG] Check: /home/simcenter/studio/Interface_Control_Example/Interface_Control_Example/Notebooks/per_source_interface_items/0000_hss_hsss_to_agss_high_pressure_helium_makeup_for_agss_charge_discharge_buffers/process_interface_items.csv
[INFO] process CSV selected: /home/simcenter/studio/Interface_Co

[]

In [4]:
from __future__ import annotations
import io
from jinja2 import Environment, BaseLoader

def _q(s: str | None) -> str:
    """Minimal YAML-safe quoting."""
    if s is None: return '""'
    s = str(s).replace("\\", "\\\\").replace('"', '\\"').replace("\n", "\\n")
    return f'"{s}"'

_PVG_APVG_TMPL = """\
- parent: !uuid {{ parent_uuid }}
  extend:
    property_value_groups:
      - name: {{ pvg_name | q }}
        promise_id: pvg_id
{% if pvg_desc %}
        description: {{ pvg_desc | q }}
{% endif %}
    applied_property_value_groups:
      - !promise pvg_id
"""

_env = Environment(loader=BaseLoader(), trim_blocks=True)   # <-- no lstrip_blocks
_env.filters["q"] = _q
_tmpl = _env.from_string(_PVG_APVG_TMPL)

def apply_pvg_decl_simple(model, parent_uuid: str, pvg_name: str, pvg_desc: str = "") -> str:
    """Render+apply declarative YAML that creates a PVG and attaches it via APVG."""
    from capellambse import decl
    yml = _tmpl.render(parent_uuid=parent_uuid, pvg_name=pvg_name, pvg_desc=pvg_desc)
    decl.apply(model, io.StringIO(yml))
    return yml



import capellambse, inspect
#print(interface)
for title, group in bundle.structural.df.groupby("title"):
    
    yml_used = apply_pvg_decl_simple(
        model,
        parent_uuid=interface.uuid,
        pvg_name="structural",
        pvg_desc="stuctural interface",
    )
    print(yml_used)  # optional: see exactly what was applied  
    
    for pvg in interface.property_value_groups :
        if pvg.name == "structural":
            update_pvg = pvg
        

    
    print(f"\n=== {title} ===")
    for _, row in group.iterrows():
        print(f"{row['property_name']}: {row['property_value']}")
        val  = row["property_value"]
        if pd.isna(val):
            val = ""
        else:
            val = str(val)
        spv = update_pvg.property_values.create('StringPropertyValue',
                                       name=row['property_name'],
                                       value=val)      
        #print(update_pvg)



   
for title, group in bundle.process.df.groupby("title"):
    yml_used = apply_pvg_decl_simple(
        model,
        parent_uuid=interface.uuid,
        pvg_name="process",
        pvg_desc="process",
    )
    print(yml_used)  # optional: see exactly what was applied    

    for pvg in interface.property_value_groups :
        if pvg.name == "process":
            update_pvg = pvg

    
    print(f"\n=== {title} ===")
    for _, row in group.iterrows():
        print(f"{row['property_name']}: {row['property_value']}")
        val  = row["property_value"]
        if pd.isna(val):
            val = ""
        else:
            val = str(val)
        spv = update_pvg.property_values.create('StringPropertyValue',
                                       name=row['property_name'],
                                       value=val)      
        #print(update_pvg)
#print(interface)

- parent: !uuid 304dad2a-1d26-46f3-a8d4-b3ae841c571b
  extend:
    property_value_groups:
      - name: "structural"
        promise_id: pvg_id
        description: "stuctural interface"
    applied_property_value_groups:
      - !promise pvg_id

=== HSS-HSSS to AGSS
High Pressure Helium Makeup for AGSS Charge/Discharge buffers ===
Internal_or_External: Internal
Subcategory: Pipe Flange
Weight: Pipe/Valve not modeled
Size: Pipe/Valve not modeled
Location: FHS Floor 4
- parent: !uuid 304dad2a-1d26-46f3-a8d4-b3ae841c571b
  extend:
    property_value_groups:
      - name: "process"
        promise_id: pvg_id
        description: "process"
    applied_property_value_groups:
      - !promise pvg_id

=== HSS-HSSS to AGSS
High Pressure Helium Makeup for AGSS Charge/Discharge buffers ===
Interface_Diameter: DN25
Interface_Elevation: 4TH FLOOR
Pipe_Specification: SC1C
Insulation_Specification: nan
Nominal_Flow_Rate: 5g/s
Maximum_Flow_Rate: TBD
Minimum_Flow_Rate: TBD
Nominal_Pressure: 6.2 Mpa
Ma

# 

Update the description with property information so that is shows up in description in SMW and in Teamcenter. Every object has a description field so this could be done for every object in a model. 


In [5]:


from __future__ import annotations
from typing import Any, Dict, List, Optional
import html
import pandas as pd
from jinja2 import Environment, BaseLoader, select_autoescape

# ---------- utilities to discover PVGs / PVs ---------------------------------

def _is_group(o):  # PropertyValueGroup?
    return o.__class__.__name__.endswith("PropertyValueGroup")

def _is_pv_def(o):  # definition, not "AppliedPropertyValue"
    n = o.__class__.__name__
    return n.endswith("PropertyValue") and not n.endswith("AppliedPropertyValue")

def _child_groups(g):
    for attr in ("owned_property_value_groups", "property_value_groups", "groups", "subgroups"):
        coll = getattr(g, attr, None)
        if coll:
            try:
                items = list(coll)
                if items: return items
            except Exception:
                pass
    try:
        return [x for x in g if _is_group(x)]
    except Exception:
        return []

def _pv_defs(g):
    for attr in ("owned_property_values", "property_values", "values"):
        coll = getattr(g, attr, None)
        if coll:
            try:
                items = list(coll)
                if items: return items
            except Exception:
                pass
    try:
        return [x for x in g if _is_pv_def(x)]
    except Exception:
        return []

def _name(o):
    return getattr(o, "long_name", None) or getattr(o, "name", None) or o.__class__.__name__

def _str_or_empty(v) -> str:
    import pandas as pd
    if v is None or (isinstance(v, float) and pd.isna(v)) or (isinstance(v, str) and v.strip().lower() in {"", "nan", "<na>", "none", "null"}):
        return ""
    return str(v)

def _effective_val(pv_def, apv_map: Dict[str, Any]) -> str:
    # Prefer APV value if present; else PV default; stringify safely
    pv_uuid = getattr(pv_def, "uuid", None)
    if pv_uuid and pv_uuid in apv_map:
        return _str_or_empty(apv_map[pv_uuid])
    return _str_or_empty(getattr(pv_def, "value", None))

def _apv_map_for_element(elem) -> Dict[str, Any]:
    out: Dict[str, Any] = {}
    acc = getattr(elem, "applied_property_values", None)
    if not acc:
        return out
    try:
        for apv in acc:
            prop = getattr(apv, "property", None)
            if prop:
                out[getattr(prop, "uuid", "")] = getattr(apv, "value", None)
    except Exception:
        pass
    return out

def _root_pvgs(elem) -> List[Any]:
    """Collect PVG roots from APVGs *and* owned PVG accessors."""
    roots: List[Any] = []
    seen: set[str] = set()

    # 1) APVG targets
    apvg_acc = getattr(elem, "applied_property_value_groups", None)
    if apvg_acc:
        try:
            for apvg in apvg_acc:
                g = getattr(apvg, "property_value_group", None)
                uid = getattr(g, "uuid", None) if g else None
                if uid and uid not in seen:
                    roots.append(g); seen.add(uid)
        except Exception:
            pass

    # 2) Owned PVGs on the element
    for attr in ("owned_property_value_groups", "property_value_groups"):
        acc = getattr(elem, attr, None)
        if not acc:
            continue
        try:
            for g in acc:
                uid = getattr(g, "uuid", None)
                if uid and uid not in seen:
                    roots.append(g); seen.add(uid)
        except Exception:
            pass

    return roots

def _attached_root_pvgs(elem):
    """All PVGs referenced by APVGs attached to elem (deduped)."""
    roots, seen = [], set()
    acc = getattr(elem, "applied_property_value_groups", None)
    print("acc",acc)
    if not acc:
        return roots
    try:
        for apvg in acc:
            g = getattr(apvg, "property_value_group", None)
            if g is None: 
                continue
            uid = getattr(g, "uuid", None)
            if uid and uid not in seen:
                roots.append(g); seen.add(uid)
    except Exception:
        pass
    return roots

def _build_group_tree(group, apv_map) -> Dict[str, Any]:
    """Recursively serialize a PVG to a dict for templating."""
    node = {
        "uuid": getattr(group, "uuid", None),
        "name": _name(group),
        "description": _str_or_empty(getattr(group, "description", "")),
        "values": [],   # list of dicts: {name, default, applied, effective}
        "children": []
    }
    for pv in _pv_defs(group):
        nm = _name(pv)
        default = _str_or_empty(getattr(pv, "value", None))
        applied = _str_or_empty(apv_map.get(getattr(pv, "uuid", ""), None))
        eff = applied if applied != "" else default
        node["values"].append({
            "name": nm,
            "default": default,
            "applied": applied,
            "effective": eff
        })
    for child in _child_groups(group):
        node["children"].append(_build_group_tree(child, apv_map))
    return node

def collect_attached_pvg_tree(elem) -> List[Dict[str, Any]]:
    """Return PVG trees (from APVGs and owned PVGs)."""
    apv_map = _apv_map_for_element(elem)
    trees = []
    for root in _root_pvgs(elem):
        trees.append(_build_group_tree(root, apv_map))
    return trees

    
# ---------- Jinja2 HTML template ---------------------------------------------

_HTML_TMPL = r"""
{% macro render_group(g) -%}
  <section class="pvg">
    <details open>
      <summary class="pvg-title">{{ g["name"] }}</summary>
      {% if g["description"] %}
      <div class="pvg-desc">{{ g["description"] }}</div>
      {% endif %}
      {% if g["values"] %}
      <table class="pv-table">
        <thead><tr><th>Property</th><th>Value</th></tr></thead>
        <tbody>
          {% for v in g["values"]|default([]) %}
          {% set tooltip = "" %}
          {% if v["applied"] and v["applied"] != v["default"] %}
            {% set tooltip = "Applied: " ~ v["applied"] ~ " | Default: " ~ v["default"] %}
          {% endif %}
          <tr title="{{ tooltip }}">
            <td class="pv-name">{{ v["name"] }}</td>
            <td class="pv-val">{{ v["effective"] }}</td>
          </tr>
          {% endfor %}
        </tbody>
      </table>
      {% endif %}
      {% for c in g["children"]|default([]) %}
        {{ render_group(c) }}
      {% endfor %}
    </details>
  </section>
{%- endmacro %}

<div class="pvg-root">
  {% if heading %}<h3 class="pvg-heading">{{ heading }}</h3>{% endif %}
  {% if trees|length == 0 %}
    <p><em>No Property Value Groups are attached.</em></p>
  {% else %}
    {% for t in trees %}
      {{ render_group(t) }}
    {% endfor %}
  {% endif %}
</div>


<style>
.pvg-root { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; line-height: 1.35; }
.pvg-heading { margin: 0.2rem 0 0.6rem 0; }
.pvg { margin: 0.4rem 0 0.6rem 0; }
.pvg-title { font-weight: 600; cursor: pointer; }
.pvg-desc { margin: 0.25rem 0 0.4rem 0; color: #444; font-size: 0.95em; }
.pv-table { border-collapse: collapse; width: 100%; margin-top: 0.3rem; }
.pv-table th, .pv-table td { border: 1px solid #ddd; padding: 4px 6px; vertical-align: top; }
.pv-name { width: 40%; font-weight: 500; }
.pv-val { width: 60%; }
</style>
"""

_env = Environment(
    loader=BaseLoader(),
    autoescape=select_autoescape(["html", "xml"]),
    trim_blocks=True
)
_tmpl = _env.from_string(_HTML_TMPL)

def render_pvg_html(elem, *, heading: str = "Properties") -> str:
    """Collect attached PVG trees and render to an HTML fragment."""
    trees = collect_attached_pvg_tree(elem)
    print('Trees:',trees)
    return _tmpl.render(trees=trees, heading=heading)

# ---------- write back to element --------------------------------------------

def set_element_description_html(elem, html_text: str, mode: str = "replace") -> None:
    """
    Set element's HTML description. Tries common attribute names.
    mode: "replace" (default) or "append".
    """
    # Try most common description attributes in Capella exports
    candidates = ["description", "summary", "desc", "documentation"]
    target_attr = None
    for attr in candidates:
        if hasattr(elem, attr):
            target_attr = attr
            break
    if target_attr is None:
        # last resort: try setting 'description' anyway
        target_attr = "description"

    if mode == "append" and getattr(elem, target_attr, None):
        current = getattr(elem, target_attr)
        html_text = (current or "") + "\n" + html_text

    setattr(elem, target_attr, html_text)


print(interface)
# elem = your Interface (or any ModelElement) that has APVGs/APVs attached
html_fragment = render_pvg_html(interface, heading="Interface Properties")
print(html_fragment)
set_element_description_html(interface, html_fragment, mode="replace")  # or mode="append"


<ComponentExchange 'hss_hsss_to_agss' (304dad2a-1d26-46f3-a8d4-b3ae841c571b)>
.allocated_exchange_items = []
.allocated_functional_exchanges = []
.allocating_physical_link = ... # backreference to PhysicalLink - omitted: can be slow to compute
.allocating_physical_paths = ... # backreference to PhysicalPath - omitted: can be slow to compute
.applied_property_value_groups = [0] <PropertyValueGroup 'structural' (4731f9d2-3b37-4a6f-8121-378280e1b768)>
                                 [1] <PropertyValueGroup 'process' (b0cc3f01-d4bc-4974-9ec5-a931924d428a)>
                                 [2] <PropertyValueGroup 'structural' (7339175a-b6da-41de-bea0-42d6a2b275ce)>
                                 [3] <PropertyValueGroup 'process' (b243b445-d185-483e-acc8-300df99b05ea)>
                                 [4] <PropertyValueGroup 'structural' (f24406ca-a8ba-49f0-b3b7-60ac0000219c)>
                                 [5] <PropertyValueGroup 'process' (d83e822b-54ad-435d-a370-e9a20061938a)>
.appli

In [6]:
model.save()
print("Done")

Done
